In [56]:
import numpy as np
import pandas as pd

In [57]:
df = pd.read_csv("odev_tenis.csv")
df

,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,yes


# Veri önişleme

Encode ederek kategorik verileri numerik verilere çevirelim.

In [58]:
play = df.iloc[:,-1].values
windy = df.iloc[:,-2].values
outlook = df.iloc[:,0].values

In [59]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [60]:
# play ve windy sütunlarında LabelEncoder kullanacağız
le = LabelEncoder()
play = le.fit_transform(play)
windy = le.fit_transform(windy)

# outlook sütununda OneHotEncoder kullanacağız
ohe = OneHotEncoder()
outlook = ohe.fit_transform(outlook.reshape(-1,1)).toarray()


In [61]:
outlookdf = pd.DataFrame(data = outlook, index= range(len(outlook)),columns=["overcast","rainy","sunny"])
playdf = pd.DataFrame(data = play, index= range(len(play)),columns= ["play"])
windydf = pd.DataFrame(data = windy, index= range(len(windy)),columns= ["windy"])

In [62]:
birlesmis = pd.concat([outlookdf,windydf,playdf,df.iloc[:,1:3],],axis=1)
birlesmis

,overcast,rainy,sunny,windy,play,temperature,humidity
0,0.0,0.0,1.0,0,0,85,85
1,0.0,0.0,1.0,1,0,80,90
2,1.0,0.0,0.0,0,1,83,86
3,0.0,1.0,0.0,0,1,70,96
4,0.0,1.0,0.0,0,1,68,80
5,0.0,1.0,0.0,1,0,65,70
6,1.0,0.0,0.0,1,1,64,65
7,0.0,0.0,1.0,0,0,72,95
8,0.0,0.0,1.0,0,1,69,70
9,0.0,1.0,0.0,0,1,75,80


# Modeli inşa etme (humidity tahmin edeceğiz)

In [63]:
X = birlesmis.drop("humidity",axis=1)
Y = birlesmis[["humidity"]]

In [64]:
from sklearn.model_selection import train_test_split

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [66]:
from sklearn.linear_model import LinearRegression

In [67]:
regressor = LinearRegression()
regressor.fit(X_train,y_train)

tahmin = regressor.predict(X_test)
tahmindf = pd.DataFrame(data = tahmin,index=range(len(tahmin)),columns=["eski model"])
y_testdf = pd.DataFrame(data = y_test.values,index=range(len(tahmin)),columns=["gerçek"] )

In [68]:
sonuc = pd.concat([tahmindf,y_testdf],axis=1)
sonuc

,eski model,gerçek
0,90.926942,80
1,69.542406,90
2,102.533821,85
3,86.054736,75
4,89.743259,70


Modelimiz ne kadar iyi? Bunun için backward elimination kullanacağız

In [69]:
import statsmodels.api as sm

In [70]:
reg_ols = np.append(arr = np.ones((len(df),1)).astype(int), values=X, axis=1)

reg_ols = reg_ols[:,[0,1,2,3,4,5,6]]
model = sm.OLS(birlesmis.iloc[:,-1],reg_ols).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               humidity   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                 -0.148
Method:                 Least Squares   F-statistic:                    0.6653
Date:                Fri, 11 Aug 2023   Prob (F-statistic):              0.661
Time:                        10:16:54   Log-Likelihood:                -49.542
No. Observations:                  14   AIC:                             111.1
Df Residuals:                       8   BIC:                             114.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.3630     35.781      1.100      0.3

/Users/macbook/anaconda3/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


4.sütunu(windy) atıyorum.

In [71]:
reg_ols = np.append(arr = np.ones((len(df),1)).astype(int), values=X, axis=1)

reg_ols = reg_ols[:,[0,1,2,3,5,6]]
model = sm.OLS(birlesmis.iloc[:,-1],reg_ols).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               humidity   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                 -0.060
Method:                 Least Squares   F-statistic:                    0.8165
Date:                Fri, 11 Aug 2023   Prob (F-statistic):              0.546
Time:                        10:16:54   Log-Likelihood:                -49.809
No. Observations:                  14   AIC:                             109.6
Df Residuals:                       9   BIC:                             112.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         28.4191     28.743      0.989      0.3

/Users/macbook/anaconda3/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


p-value değerlerine baktığımızda model kısmen kabul edilebilir.

# Modeli tekrar oluşturma

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X.drop("windy",axis=1), Y, test_size=0.33, random_state=42)

In [73]:
regressor.fit(X_train,y_train)

tahmin2 = regressor.predict(X_test)
tahmindf2 = pd.DataFrame(data = tahmin2,index=range(len(tahmin)),columns=["yeni model"])
y_testdf = pd.DataFrame(data = y_test.values,index=range(len(tahmin)),columns=["gerçek"] )


In [74]:
yeni_sonuc = pd.concat([tahmindf,tahmindf2,y_testdf],axis=1)
yeni_sonuc

,eski model,yeni model,gerçek
0,90.926942,89.871209,80
1,69.542406,74.147373,90
2,102.533821,97.076463,85
3,86.054736,82.263135,75
4,89.743259,92.668091,70
